In [1]:
import pandas as pd


from automlwrapper import AutoMLWrapper


/home/max/miniconda3/envs/AutoML/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-21 17:27:36.507007: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-21 17:27:37.069370: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-11-21 17:27:37.069445: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugi

Using TensorFlow backend


In [2]:
df = pd.read_csv('data/glass.csv')

In [3]:
wrapper = AutoMLWrapper('autogluon')

wrapper.set_out_path('test/ag')

#wrapper.initialize(df, 'Type', task_type='classification', data_type='tabular', problem_type='multiclass')

wrapper.train(data=df, target_column='Type', task_type='classification', data_type='tabular', problem_type='multiclass')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:7                                                                                    │
│                                                                                                  │
│   4                                                                                              │
│   5 #wrapper.initialize(df, 'Type', task_type='classification', data_type='tabular', problem     │
│   6                                                                                              │
│ ❱ 7 wrapper.train(data=df, target_column='Type', task_type='classification', data_type='tabu     │
│   8                                                                                              │
│                                                                                                  │
│ /home/max/miniconda3/envs/AutoML/lib/python3.8/site-packages/automlwrapper/AutoMLWrapper.py:66   │
│ in train                                                                                         │
│                                                                                                  │
│   63 │   def train(self, data: Any, target_column: str, task_type: Optional[str] = None, data    │
│   64 │   │   │     problem_type: Optional[str] = None, hyperparameters: Optional[Dict[str, An    │
│   65 │   │   if not self.is_initialized:                                                         │
│ ❱ 66 │   │   │   self.initialize(data, target_column, task_type, data_type, problem_type)        │
│   67 │   │                                                                                       │
│   68 │   │   self.library._train_model(data, target_column, hyperparameters)                     │
│   69                                                                                             │
│                                                                                                  │
│ /home/max/miniconda3/envs/AutoML/lib/python3.8/site-packages/automlwrapper/AutoMLWrapper.py:31   │
│ in initialize                                                                                    │
│                                                                                                  │
│   28 │   │   │   self.library = AutoKerasWrapper()                                               │
│   29 │   │                                                                                       │
│   30 │   │   elif self.library_name == "autogluon":                                              │
│ ❱ 31 │   │   │   self.library = AutoGluonWrapper()                                               │
│   32 │   │                                                                                       │
│   33 │   │   else:                                                                               │
│   34 │   │   │   raise ValueError("Invalid library name")                                        │
│                                                                                                  │
│ /home/max/miniconda3/envs/AutoML/lib/python3.8/site-packages/automlwrapper/AutoGluon/AutoGluonWr │
│ apper.py:14 in __init__                                                                          │
│                                                                                                  │
│    11 │   #-----------------------------------------------------------------------------------   │
│    12 │   def __init__(self, **kwargs):                                                          │
│    13 │   │   super().__init__(**kwargs)                                                         │
│ ❱  14 │   │   self.config = AutoGluonConfig(os.path.join(os.path.dirname(__file__), 'AutoGluon   │
│    15 │   │   self.output_path = os.path.join(os.path.dirname(__file__),                         │
│    16 │   │   │   │   │   │   │   │   │   │    f'../output/